In [0]:
dbutils.fs.mount(source = "wasbs://rafidata@adlsstrgdb.blob.core.windows.net",mount_point = "/mnt/silv",
extra_configs ={"fs.azure.account.key.adlsstrgdb.blob.core.windows.net":'hkxruKf48RSxljlC3ILBk7KAfSsGTswA66gTVNZu8js2jta3Gbk7Dfoyd6s/hX3e2yEx6AWjPuyN+ASt4ckOWQ=='})

Out[3]: True

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([\
                    StructField("id", StringType(),True),\
                    StructField("fname", StringType(),True),\
                    StructField("lname", StringType(),True),\
                    StructField("mobile", StringType(),True),\
                    StructField("home", StringType(),True),\
                    StructField("Address", StringType(),True),\
                    StructField("city", StringType(),True),\
                    StructField("state", StringType(),True),\
                    StructField("Zip_code", StringType(),True)])

In [0]:
scenario_1 = spark.read.csv('dbfs:/mnt/silv/data-files/data-files/customers-tab-delimited/part-m-00000',sep="\t",schema=schema)
scenario_1.show()

+---+-----------+---------+---------+---------+--------------------+-------------+-----+--------+
| id|      fname|    lname|   mobile|     home|             Address|         city|state|Zip_code|
+---+-----------+---------+---------+---------+--------------------+-------------+-----+--------+
|  1|    Richard|Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|  Brownsville|   TX|   78521|
|  2|       Mary|  Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|    Littleton|   CO|   80126|
|  3|        Ann|    Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|       Caguas|   PR|   00725|
|  4|       Mary|    Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common|   San Marcos|   CA|   92069|
|  5|     Robert|   Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|       Caguas|   PR|   00725|
|  6|       Mary|    Smith|XXXXXXXXX|XXXXXXXXX|3151 Sleepy Quail...|      Passaic|   NJ|   07055|
|  7|    Melissa|   Wilcox|XXXXXXXXX|XXXXXXXXX|9453 High Concession|       Caguas|   PR|   00725|
|  8|      Megan|   

In [0]:
from pyspark.sql.functions import *

#from pyspark.sql.functions import concat

# show the client info who is in california
scenario_01 = scenario_1.filter(scenario_1.state == 'CA').withColumn('full_name',concat(scenario_1.lname,lit(' '),scenario_1.lname))
scenario_001 = scenario_01.select('id','full_name','Address','city','state','Zip_code')
#final ouput in text format
result = scenario_001.withColumn('output',concat(scenario_001.id,lit(' '),\
                             scenario_001.full_name,lit(' '),\
                             scenario_001.Address,lit(' '),\
                             scenario_001.city,lit(' '),\
                             scenario_001.state,lit(' '),\
                             scenario_001.Zip_code,lit(' '))).select('output')
result.write.option('sep','\t').mode('overwrite').text("/mnt/silv/result/scenario1/solution/")
result.show()
#save in result/scenario1/solution 
#only with code CA
#full name in a column
# scenario_001.coalesce(1).write.format("text").option("header", "false").mode("append").save("/mnt/silv/result/scenario1/solution/output.txt")


+--------------------+
|              output|
+--------------------+
|4 Jones Jones 832...|
|14 Smith Smith 56...|
|15 Luna Luna 673 ...|
|18 Smith Smith 27...|
|35 Wright Wright ...|
|40 Smith Smith 73...|
|44 Smith Smith 13...|
|50 Kim Kim 938 Ru...|
|59 James James 23...|
|70 Simmons Simmon...|
|72 Gillespie Gill...|
|76 Young Young 76...|
|89 Smith Smith 91...|
|106 Freeman Freem...|
|114 Warner Warner...|
|115 Smith Smith 1...|
|125 Gallagher Gal...|
|139 Maxwell Maxwe...|
|149 Mcclain Mccla...|
|156 Smith Smith 2...|
+--------------------+
only showing top 20 rows



##TASK -->> 2.3

In [0]:
%fs ls dbfs:/mnt/silv/data-files/data-files/orders_parquet/

path,name,size,modificationTime
dbfs:/mnt/silv/data-files/data-files/orders_parquet/741ca897-c70e-4633-b352-5dc3414c5680.parquet,741ca897-c70e-4633-b352-5dc3414c5680.parquet,488255,1675075237000


In [0]:
order = spark.read.parquet('dbfs:/mnt/silv/data-files/data-files/orders_parquet/741ca897-c70e-4633-b352-5dc3414c5680.parquet')
order.show()

+--------+-------------+-----------------+---------------+
|order_id|   order_date|order_customer_id|   order_status|
+--------+-------------+-----------------+---------------+
|       1|1374710400000|            11599|         CLOSED|
|       2|1374710400000|              256|PENDING_PAYMENT|
|       3|1374710400000|            12111|       COMPLETE|
|       4|1374710400000|             8827|         CLOSED|
|       5|1374710400000|            11318|       COMPLETE|
|       6|1374710400000|             7130|       COMPLETE|
|       7|1374710400000|             4530|       COMPLETE|
|       8|1374710400000|             2911|     PROCESSING|
|       9|1374710400000|             5657|PENDING_PAYMENT|
|      10|1374710400000|             5648|PENDING_PAYMENT|
|      11|1374710400000|              918| PAYMENT_REVIEW|
|      12|1374710400000|             1837|         CLOSED|
|      13|1374710400000|             9149|PENDING_PAYMENT|
|      14|1374710400000|             9842|     PROCESSIN

In [0]:
from pyspark.sql.functions import from_utc_timestamp
from pyspark.sql.functions import *

# from pyspark.sql.types import StringType
orders = order.filter(order.order_status == 'COMPLETE').withColumn('order_date_time',from_utc_timestamp(from_unixtime(order.order_date/1000,"yyyy-MM-dd hh:mm:ss"),'GMT+5')).select('order_id','order_date_time','order_status')
orders.show()

+--------+-------------------+------------+
|order_id|    order_date_time|order_status|
+--------+-------------------+------------+
|       3|2013-07-25 17:00:00|    COMPLETE|
|       5|2013-07-25 17:00:00|    COMPLETE|
|       6|2013-07-25 17:00:00|    COMPLETE|
|       7|2013-07-25 17:00:00|    COMPLETE|
|      15|2013-07-25 17:00:00|    COMPLETE|
|      17|2013-07-25 17:00:00|    COMPLETE|
|      22|2013-07-25 17:00:00|    COMPLETE|
|      26|2013-07-25 17:00:00|    COMPLETE|
|      28|2013-07-25 17:00:00|    COMPLETE|
|      32|2013-07-25 17:00:00|    COMPLETE|
|      35|2013-07-25 17:00:00|    COMPLETE|
|      45|2013-07-25 17:00:00|    COMPLETE|
|      56|2013-07-25 17:00:00|    COMPLETE|
|      63|2013-07-25 17:00:00|    COMPLETE|
|      65|2013-07-25 17:00:00|    COMPLETE|
|      67|2013-07-25 17:00:00|    COMPLETE|
|      71|2013-07-25 17:00:00|    COMPLETE|
|      72|2013-07-25 17:00:00|    COMPLETE|
|      76|2013-07-25 17:00:00|    COMPLETE|
|      80|2013-07-25 17:00:00|  

In [0]:
orders.write.option("compression","gzip").json("/mnt/silv/result/scenario2/solution/")


###TASK 3

In [0]:
scenario_3 = spark.read.csv('dbfs:/mnt/silv/data-files/data-files/customers-tab-delimited/part-m-00000',sep="\t",schema=schema)
scenario_3.show()

+---+-----------+---------+---------+---------+--------------------+-------------+-----+--------+
| id|      fname|    lname|   mobile|     home|             Address|         city|state|Zip_code|
+---+-----------+---------+---------+---------+--------------------+-------------+-----+--------+
|  1|    Richard|Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|  Brownsville|   TX|   78521|
|  2|       Mary|  Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|    Littleton|   CO|   80126|
|  3|        Ann|    Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|       Caguas|   PR|   00725|
|  4|       Mary|    Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common|   San Marcos|   CA|   92069|
|  5|     Robert|   Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|       Caguas|   PR|   00725|
|  6|       Mary|    Smith|XXXXXXXXX|XXXXXXXXX|3151 Sleepy Quail...|      Passaic|   NJ|   07055|
|  7|    Melissa|   Wilcox|XXXXXXXXX|XXXXXXXXX|9453 High Concession|       Caguas|   PR|   00725|
|  8|      Megan|   

In [0]:
scenario_03 = scenario_3.filter(scenario_3.city == 'Caguas')

In [0]:
scenario_03.coalesce(1).write.format('orc').option("compression","snappy").save("/mnt/silv/result/scenario3/solution/output.orc")

###TASK 4

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

sqema = StructType([\
                    StructField("id", IntegerType(),False),\
                    StructField("matches_won", IntegerType(),True),\
                    StructField("sports", StringType(),True)\
                    ])
scenario_4 = spark.read.csv('dbfs:/mnt/silv/data-files/data-files/categories/part-m-00000',schema=sqema)

scenario_4.write.mode('overwrite').option("compression","lz4").csv("/mnt/silv/result/scenario4/solution/")

##TASK 5

In [0]:
scenario_5 = spark.read.format("avro").load('dbfs:/mnt/silv/data-files/data-files/products_avro/*')
scenario_5.show()

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|      1009|                 45|Diamond Fear No E...|                   |       599.99|http://images.acm...|
|      1010|                 46|DBX Vector Series...|                   |        19.98|http://images.acm...|
|      1011|                 46|Old Town Canoe Sa...|                   |       499.99|http://images.acm...|
|      1012|                 46|Pelican Trailblaz...|                   |       299.99|http://images.acm...|
|      1013|                 46|Perception Sport ...|                   |       349.99|http://images.acm...|
|      1014|                 46|O'Brien Men's Neo...|                   |        49.98|http://images.acm...|
|      1015|       

In [0]:
scenario_05 = scenario_5.filter(scenario_5.product_price >= 1000.0)
scenario_05.write.mode('overwrite').option("compression","snappy").parquet("/mnt/silv/result/scenario5/solution/")

##TASK 6

In [0]:
scenario_6 = spark.read.format('avro').load('dbfs:/mnt/silv/data-files/data-files/products_avro/*')
scenario_06 =  scenario_6.filter(scenario_6.product_price >= 1000.0).filter(scenario_6.product_name.contains('Treadmill'))
scenario_06.show(truncate=False)
scenario_06.write.mode('overwrite').option("compression","gzip").parquet("/mnt/silv/result/scenario6/solution/")

+----------+-------------------+------------------+-------------------+-------------+--------------------------------------------------+
|product_id|product_category_id|product_name      |product_description|product_price|product_image                                     |
+----------+-------------------+------------------+-------------------+-------------+--------------------------------------------------+
|496       |22                 |SOLE F85 Treadmill|                   |1799.99      |http://images.acmesports.sports/SOLE+F85+Treadmill|
|66        |4                  |SOLE F85 Treadmill|                   |1799.99      |http://images.acmesports.sports/SOLE+F85+Treadmill|
|199       |10                 |SOLE F85 Treadmill|                   |1799.99      |http://images.acmesports.sports/SOLE+F85+Treadmill|
+----------+-------------------+------------------+-------------------+-------------+--------------------------------------------------+



##TASK 7

In [0]:
from pyspark.sql.functions import from_utc_timestamp,from_unixtime

scenario_7 = spark.read.parquet('dbfs:/mnt/silv/data-files/data-files/orders_parquet/741ca897-c70e-4633-b352-5dc3414c5680.parquet')
# date in yyy mm  dd 
scenario_07 = scenario_7.withColumn('order_date',from_utc_timestamp(from_unixtime(scenario_7.order_date/1000,"yyyy-MM-dd"),'GMT+5')) 
# output all pending orders in jul 2013
scenario_007 = scenario_07.filter((scenario_07.order_date > '2013-07-01 00:00:00') &(scenario_07.order_date < '2013-07-31 00:00:00')).filter(scenario_07.order_status.contains('PENDING_PAYMENT')).select('order_date','order_status')
# json format snappy with order date and status
scenario_007.write.mode('overwrite').option("compression","snappy").json("/mnt/silv/result/scenario7/solution/")